In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
from ipywidgets import interactive
import image
import mapa

import numpy as np

import glob
from collections import Counter

In [ ]:
%load_ext autoreload
%autoreload 2

# Load image, resize and convert

In [ ]:
img = cv.imread('./images/walls.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)

In [ ]:
calibration = np.load('rig/calib.npz')

In [ ]:
mtx = calibration['mtx']
dist = calibration['dist']
rvecs = calibration['rvecs']
tvecs = calibration['tvecs']

In [ ]:
h, w = img.shape[:2]

In [ ]:
newCameraMtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

In [ ]:
undistortedImg = cv.undistort(img, mtx, dist, None, newCameraMtx)

In [ ]:
plt.imshow(undistortedImg)

Plot histograms of color spaces, both rgb and hsv

# Color tweaking

Find parameters for extracting the mask (high and low colors for range limits) using sliders and looking for the best match.

In [ ]:
hsv_img = cv.cvtColor(img, cv.COLOR_RGB2HSV)
def mask(h, sl, vl, sh, vh):
    m = cv.inRange(
        hsv_img,
        (h-10, sl, vl),
        (h+10, sh, vh),
    )
    plt.imshow(m)

In [ ]:
interactive(
    mask,
    h=(0+10, 179-10),
    sl=(0, 255),
    vl=(0, 255),
    sh=(0, 255),
    vh=(0, 255),
)

# Create the mask

In [ ]:
BLUE_LOWER = (100, 144, 144)
BLUE_UPPER = (120, 255, 255)

# ROI corner finder

Find maze corners in specific regions of interest

 method reference; to index tPlot corners to a new image

In [ ]:
image_class = image.Image(undistortedImg)
image_class.set_areas(
    image.Area(50, 150),
    image.Area(350, 450),
    image.Area(150, 250),
    image.Area(450, 550),
)
image_class.set_origin_points((0, 99), (0, 99))

In [ ]:
plt.imshow(image_class.img)

In [ ]:
hsv_class = image_class.rgb_to_hsv()

In [ ]:
plt.imshow(hsv_class.img)

In [ ]:
mask_class = hsv_class.make_mask(BLUE_LOWER, BLUE_UPPER)

In [ ]:
plt.imshow(mask_class.img)

In [ ]:
corners = mask_class.find_corners()

In [ ]:
corners

In [ ]:
debug_class = image_class.copy()

In [ ]:
debug_class.paint_corners(corners)

In [ ]:
plt.imshow(debug_class.img)

In [ ]:
fix1_class = image_class.fix_perspective(corners, (10, 10), (24, 24))

In [ ]:
fix1_class.set_areas(
    image.Area(0, 50),
    image.Area(250, 300),
    image.Area(0, 50),
    image.Area(250, 300),
)
fix1_class.set_origin_points(
    (0, 49),
    (0, 49),
)

In [ ]:
plt.imshow(fix1_class.img)

In [ ]:
plt.imshow(fix1_class.img[10:19, 0:9])

In [ ]:
fix1_mapa = mapa.Mapa.from_image(fix1_class.img, (10, 10), (24, 24))

In [ ]:
map_image = image.Image.from_map(fix1_mapa)

In [ ]:
print(fix1_class.img.dtype, fix1_class.img.shape)
#print(type(map_image.img))
print(map_image.img.dtype, map_image.img.shape)

In [ ]:
plt.imshow(map_image.img)
plt.savefig('map.jpg')

In [ ]:
Counter

In [ ]:
kmeans_whole = np.float32(fix1_class.img.reshape((-1, 3)))
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
n_clusters = 6
ret, label, center = cv.kmeans(kmeans_whole, n_clusters, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

In [ ]:
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((fix1_class.img.shape))

In [ ]:
plt.imshow(res2)

In [ ]:
# WIP: Not for use
crop_img = fix1_class.img[10*2:10*3-1, 10*2:10*3-1]
kmeans_img = np.float32(crop_img.reshape((-1, 3)))
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
n_clusters = 1
ret,label,center=cv.kmeans(kmeans_img,n_clusters,None,criteria,10,cv.KMEANS_RANDOM_CENTERS)

In [ ]:
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((crop_img.shape))

In [ ]:
center

In [ ]:
plt.imshow(res2)